In [ ]:
import os  # 운영 체제와 상호작용하기 위한 os 모듈 임포트
import tempfile  # 임시 파일 생성을 위한 tempfile 모듈 임포트
import logging  # 로그 메시지를 위한 logging 모듈 임포트
from flask import Flask, request, jsonify  # 웹 애플리케이션 생성을 위한 Flask와 유틸리티 임포트
import whisper  # Whisper ASR 모델 임포트
import torch  # 텐서 연산을 위한 PyTorch 임포트
import threading  # 스레드를 생성하기 위한 threading 모듈 임포트
import queue  # 작업 큐 생성을 위한 queue 모듈 임포트
import uuid  # 고유한 작업 ID 생성을 위한 uuid 모듈 임포트

# 로그 메시지를 info 레벨로 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)  # 이 모듈을 위한 로거 생성

app = Flask(__name__)  # Flask 애플리케이션 인스턴스 생성

# 작업 큐와 결과 저장을 위한 딕셔너리 초기화
task_queue = queue.Queue()
results = {}

# 오디오 파일을 처리할 작업자 스레드 수 설정
num_worker_threads = 4

# 오디오 파일을 처리할 작업자 함수 정의
def worker(thread_id):
    logger.info(f"Worker {thread_id} starting, loading Whisper model...")
    whisper_model = whisper.load_model("medium")  # Whisper ASR 모델 로드
    logger.info(f"Worker {thread_id} loaded Whisper model.")

    while True:  # 작업자가 계속 실행되도록 무한 루프
        task_id, audio_path = task_queue.get()  # 큐에서 작업 가져오기
        if task_id is None:  # 종료 신호 확인
            break  # 종료 신호를 받으면 루프 종료
        
        logger.info(f"Worker {thread_id} processing task {task_id}")
        
        try:
            # Whisper 모델을 사용하여 오디오 파일을 전사
            results[task_id] = {"status": "processing"}
            result = whisper_model.transcribe(audio_path)
            # segment에서 id, start, end, text 
            # 비동기 
            # API통신.uri(result, "spring url")
            
            
            results[task_id] = {"status": "completed", "result": result}  # 결과를 딕셔너리에 저장
        except Exception as e:
            logger.error(f"Worker {thread_id} encountered an error: {str(e)}")  # 발생한 오류 로그
            results[task_id] = {"status": "failed", "error": str(e)}  # 오류를 결과에 저장
        finally:
            os.remove(audio_path)  # 임시 오디오 파일 제거
            if torch.cuda.is_available():  # CUDA 사용 가능 여부 확인
                torch.cuda.empty_cache()  # 필요 시 CUDA 메모리 캐시 지우기
        
        task_queue.task_done()  # 작업 완료 표시

# 작업자 스레드 시작
threads = []
for i in range(num_worker_threads):
    t = threading.Thread(target=worker, args=(i,))  # 작업자 함수에 대한 새 스레드 생성
    t.start()  # 스레드 시작
    threads.append(t)  # 스레드 목록에 추가

@app.route('/', methods=['GET'])  # 홈 페이지를 위한 라우트 정의
def home():
    return "Welcome to the Asynchronous Flask Server with Whisper!"  # 환영 메시지 반환

@app.route('/stt', methods=['POST'])  # STT 요청을 위한 라우트 정의
def stt_request():
    logger.info("Received STT request")
    
    if 'audio' not in request.files:  # 오디오 파일이 제공되었는지 확인
        logger.warning("No audio file provided in the request")
        return jsonify({'error': 'No audio file provided'}), 400  # 제공되지 않으면 오류 반환
    
    audio_data = request.files['audio']  # 요청에서 오디오 파일 가져오기
    
    if audio_data.filename == '':  # 파일 이름이 비어 있는지 확인
        logger.warning("Empty filename provided")
        return jsonify({'error': 'No selected file'}), 400  # 비어 있으면 오류 반환
    
    logger.info(f"Processing audio file: {audio_data.filename}")
    
    # 오디오 데이터를 저장할 임시 파일 생성
    with tempfile.NamedTemporaryFile(delete=False) as temp_audio:
        audio_data.save(temp_audio.name)  # 오디오 데이터를 임시 파일에 저장
        temp_audio_path = temp_audio.name  # 임시 파일 경로 저장
    
    # 고유한 작업 ID를 생성하고 작업을 큐에 추가
    task_id = str(audio_data.filename)  # 작업을 위한 고유 식별자 생성
    task_queue.put((task_id, temp_audio_path))  # 큐에 작업 추가
    
    logger.info(f"Task {task_id} added to the queue")
    
    return jsonify({"task_id": task_id, "status": "processing"}), 202  # 작업 ID와 상태 반환

@app.route('/stt_result/<task_id>', methods=['GET'])  # 작업 상태를 확인하기 위한 라우트 정의
def get_status(task_id):
    if task_id not in results:  # 작업 ID가 결과에 없으면
        return jsonify({"status": "No_result"}), 404  # 처리 중 상태 반환
    
    result = results[task_id]  # 주어진 작업 ID에 대한 결과 가져오기
    
    if result["status"] == "completed":  # 작업이 완료되었는지 확인
        del results[task_id]  # 메모리 해제를 위해 결과 딕셔너리에서 제거
        return jsonify(result), 200  # 완료된 결과 반환
    elif result["status"] == "failed":  # 작업이 실패했는지 확인
        del results[task_id]  # 메모리 해제를 위해 결과 딕셔너리에서 제거
        return jsonify(result), 500  # 실패한 결과 반환
    else:
        return jsonify({"status": "processing"}), 202  # 여전히 처리 중인 상태 반환

@app.errorhandler(500)  # 500 내부 서버 오류 처리
def internal_error(error):
    logger.error(f"Internal Server Error: {str(error)}")  # 오류 로그
    return jsonify({"error": "Internal Server Error"}), 500  # 오류 메시지 반환

if __name__ == '__main__':
    try:
        app.run(host='0.0.0.0', port=5000)  # 포트 5000에서 Flask 앱 시작
    finally:
        # 애플리케이션 종료 시 작업자 스레드 중지
        torch.cuda.empty_cache()  # 메모리 캐시 비우기
        for _ in range(num_worker_threads):
            task_queue.put((None, None))  # 스레드에 중지 신호 전송
        for t in threads:
            t.join()  # 모든 스레드가 종료될 때까지 대기
